In [6]:
import pandas as pd
import os
!pip install google-cloud-bigquery
from google.cloud import bigquery
import numpy as np

In [10]:
cServico = r"C:\Users\User\OneDrive\Villam\POWERBI\Cnaes\Atividades_vs_CNAE_SN.csv"
cServico = pd.read_csv(cServico, delimiter=';',encoding='ansi', dtype=str)
ccnpj = r"C:\Users\User\OneDrive\Villam\POWERBI\Cnaes\F.K03200$Z.D30311.CNAECSV"
ccnpj = pd.read_csv(ccnpj, delimiter=';',encoding='ansi', header = None, names=['CNAE', 'Descrição'])
dimCnae = r"C:\Users\User\OneDrive\Villam\POWERBI\Cnaes\DIM_CNAES.xlsx"
dimCnae = pd.read_excel(dimCnae, dtype=str)


In [11]:
condicoes = [
    cServico['Anexo'] == "I",
    cServico['Anexo'] == "II"
]

valores = ['Comercio', 'Industria']

# Adiciona a nova coluna 'tipo'
cServico['tipo'] = np.select(condicoes, valores, default='Serviço')

cServico = cServico.rename(columns={
    'CNAE': 'cnae',
    'Descrição': 'descricao',
    'Anexo': 'anexo',
    'tipo': 'tipo'  # Se quiser mudar para outro nome, altere aqui
})

cServico = cServico.drop_duplicates(subset='cnae')

cServico

,cnae,descricao,anexo,tipo
0,9900800,Organismos internacionais e outras instituiçõe...,IV,Serviço
1,9700500,Serviços domésticos,IV,Serviço
2,9609299,Outras atividades de serviços pessoais não esp...,III,Serviço
3,9609208,Higiene e embelezamento de animais domésticos,III,Serviço
4,9609207,Alojamento de animais domésticos,III,Serviço
...,...,...,...,...
1249,112101,Cultivo de algodão herbáceo,I,Comercio
1250,111399,Cultivo de outros cereais não especificados an...,I,Comercio
1251,111303,Cultivo de trigo,I,Comercio
1252,111302,Cultivo de milho,I,Comercio


In [12]:
# 1 = comercio
# 2 = industria
# 3 e 4 = serviço

client = bigquery.Client.from_service_account_json(r"C:\Users\User\Documents\GitHub\infra-itaborai-ce9c0b4d25c8.json")
# Definindo o projeto e o dataset
project = 'infra-itaborai'
dataset_id = 'mds_cnpj'
dataset_ref = client.dataset(dataset_id)

# subindo 
cServico = cServico
table_ref = dataset_ref.table('cnae_servico')


# Carregando 
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

job = client.load_table_from_dataframe(cServico, table_ref, job_config=job_config)

job.result()  # Esperando





LoadJob<project=infra-itaborai, location=us-central1, id=4adbc708-84b5-4741-bce5-e537ba7c972d>

In [ ]:
cGeralDim = pd.merge(cServico,dimCnae, left_on='CNAE',right_on='Subclasse', how = 'outer')
cGeralDim

In [ ]:
# cGeral = pd.merge(ccnpj,cServico, on = 'CNAE', how = 'outer', suffixes=('_CNPJ', '_SERVICO'))
# cGeral

In [ ]:
cServico

In [ ]:
from google.cloud import bigquery
import pandas as pd



In [ ]:


# Consulta SQL para carregar dados de outra conta
query = """
    SELECT * FROM `villam-data-science.dm_itaborai.nfse_concat` 
    WHERE SISTEMA = 'giss'
"""

# Executa a consulta e carrega os dados em um DataFrame
cServico = client.query(query).to_dataframe()


# Define o projeto e o dataset de destino
project = 'infra-itaborai'
dataset_id = 'teste2'
table_id = 'notastestes'  # Nome da tabela de destino

# Referência completa para a tabela de destino
table_ref = f"{project}.{dataset_id}.{table_id}"

# Configuração do job de carga para sobrescrever a tabela de destino
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# Carrega o DataFrame para a tabela de destino
job = client.load_table_from_dataframe(cServico, table_ref, job_config=job_config)
job.result()  # Aguarda a conclusão do job

print("Dados carregados com sucesso para a tabela de destino.")

In [ ]:
# Autenticação
client = bigquery.Client.from_service_account_json(r"C:\Users\nelio\Downloads\infra-itaborai-8632346554bf.json")

In [ ]:
client